<img style="float: right; padding: 10px;" width="200" src="images/logo-onem2m-the-iot-standard.jpg">

# oneM2M - Basic Resources and Requests
This notebook shows the basic interactions with a CSE using REST calls. Examples include:

- Create an &lt;AE> resource
- Create a &lt;container> resource
- Create one or more &lt;contentInstance> resources
- Retrieve the latest &lt;contentInstance> resource
- Update the &lt;container> resource
- Retrieve the &lt;container> resource
- Delete the &lt;container> resource


## Intitialization
The section does import necessary modules and configurations, and prepares the CSE for this notebook.

In [ ]:
%run src/init.py basic

## Register an &lt;AE> Resource

<img align="right" src="images/02/register_AE.png">


This example creates a new &lt;AE> resource in the CSE. &lt;AE>'s represent applications or services.

Creating this (or other) resource is done using a POST request and with providing a couple of mandatory attributes:

- **api** : Application Identifier. An Application Identifier uniquely identifies an M2M Application in a given context.
- **rr** : Request Reachability. This attribute indicates whether a resource can receive requests.
- **srv** : Supported Release Versions. An array that specifies the supported oneM2M specification releases.

 
<p>
<div style="background-color:transparent;border:2px #b42025 solid;padding: 10px;width: calc(100% - 350px);">
    <div style="color:#b42025;"><b><i class="fa fa-info-circle" aria-hidden="true"></i>&nbsp; oneM2M</b></div>
    <div style="color:#545054;">
        The request target is the &lt;CSEBase> resource. All create requests target a parent resource.
    </div>
</div>
</p>



In [ ]:
CREATE (                                                     # CREATE request
    
    # Create the AE resource under the CSEBase
    to                      = cseBaseName,  
    
    # Request Parameters
    originator              = 'Cmyself',                     # Assign an originator ID, must start with 'C'
    requestIdentifier       = '123',                         # Unique request identifier
    releaseVersionIndicator = '3',                           # Release version indicator
    resourceType            = Type.AE,                       # Type of the resource: AE
    
    # Request Body
    primitiveContent = 
        {   
            'm2m:ApplicationEntity': {
                'resourceName':             'Notebook-AE',   # Name of the resource 
                'App-ID':                   'NnotebookAE',   # Application ID, must start with 'N'
                'requestReachability':      True,            # AE can receive requests
                'supportedReleaseVersions': [ '3' ]          # Supports oneM2M release 3
            }
        }
)

The response introduces some new attributes:

- **pi** : This is the identifier of the parent resource.
- **et** : Expiration time/date after which the CSE will delete the resource.
- **aei** : An Application Entity Identifier uniquely identifies an AE globally.

<img align="right" src="images/02/create_container.png">

## Create a &lt;container> Resource
In this section we add a &lt;container> resource to the &lt;AE> resource. A &lt;container> represents a structure in the oneM2M resource tree that can have a configurable number of data instances as child resources. A &lt;container> may also have sub-containers.

If created with no further attributes, the CSE will assign meaningful defaults to the &lt;container> resource's attributes.

In [ ]:
CREATE (                                      # CREATE request
    
    # Create the container resource under the AE
    to                      = cseBaseName + '/Notebook-AE', 

    # Request Parameters
    originator              = 'Cmyself',      # Set the originator
    requestIdentifier       = '123',          # Unique request identifier
    releaseVersionIndicator = '3',            # Release version indicator
    resourceType            = Type.Container, # Type of the resource: container
    
    # Request Body
    primitiveContent = 
        {
            'm2m:Container': {
                'resourceName': 'Container'   # Set the resource name
            }
        }
)


<p>
<div  style="background-color:transparent;border:2px #005480 solid;padding: 10px;width: calc(100% - 350px)">
    <div style="color:#005480;"><b><i class="fa fa-info-circle" aria-hidden="true"></i>&nbsp; Note</b></div>
    <div style="color:#545054;">
        If you see an error "409" or "Name already present" then don't worry. It just means that an &lt;container> resource with the same name already exists in the CSE, perhaps from a previous run of this notebook cell.    
    </div>
</div>
</p>

The following attributes are used with the &lt;Container> resource:

- **st** : The State Tag is incremented every time the resource is modified.
- **mni** : Maximum number of direct data instances in the &lt;Container> resource.
- **mbs** : Maximum size in bytes of data.
- **mia** : Maximum age of a direct data instances in the &lt;Container> resource.
- **cni** : Current number of direct data instances in the &lt;Container> resource.
- **cbs** : Current size in bytes of data.
- **ol** : Resource identifier of a virtual resource that points to the oldest data instance of the &lt;Container> resource.
- **la** : Resource identifier of a virtual resource that points to the latest data instance of the &lt;Container> resource.

<p>
<div style="background-color:transparent;border:2px #b42025 solid;padding: 10px;width: calc(100% - 350px);">
    <div style="color:#b42025;"><b><i class="fa fa-info-circle" aria-hidden="true"></i>&nbsp; oneM2M</b></div>
    <div style="color:#545054;">
        A &lt;container> has two virtual child resources, <i>latest</i> (la) and <i>oldest</i> (ol), which virtually represent the latest and oldest &lt;contentInstance> resources, respectively.
    </div>
</div>
</p>

## Add a &lt;contentInstance> to the &lt;container>

<img align="right" src="images/02/create_contentInstance.png">

Now, we add some data to the *myContainer* &lt;container>. 

These attributes are part of the request:

- **cnf** : This specifies the content format. It specifies the media type as well as an encoding type.
- **con** : This is the actual content (ie. the value) that will be stored in the &lt;Container resource. It must contain media information and may optionally specify an optional content encoding (here 0 means "plain, no transfer encoding"), and content security.

&lt;contentInstance> resources can only be added and read, but not updated or deleted.


<p>
<div style="background-color:transparent;border:2px #b42025 solid;padding: 10px;width: calc(100% - 350px);">
    <div style="color:#b42025;"><b><i class="fa fa-info-circle" aria-hidden="true"></i>&nbsp; oneM2M</b></div>
    <div style="color:#545054;">
In the following request we don't provide a resource. Instead, the CSE provides one for the newly created resource.
    </div>
</div>
</p>

<p>
<div  style="background-color:transparent;border:2px #005480 solid;padding: 10px;width: calc(100% - 350px)">
    <div style="color:#005480;"><b><i class="fa fa-info-circle" aria-hidden="true"></i>&nbsp; Note</b></div>
    <div style="color:#545054;">
        You can execute the following code as often as you like to create more &lt;contentInstance> resources.
    </div>
</div>
</p>


In [ ]:
CREATE (                                            # CREATE request

    # Create the content instance resource under the container
    to                      = cseBaseName + '/Notebook-AE/Container', 

    # Request Parameters
    originator              = 'Cmyself',            # Set the originator
    requestIdentifier       = '123',                # Unique request identifier
    releaseVersionIndicator = '3',                  # Release version indicator
    resourceType            = Type.ContentInstance, # Type of the resource: contentInstance
    
    # Request Body
    primitiveContent = 
        {
            'm2m:ContentInstance': {
                'contentInfo': 'text/plain:0',      # Media type of the content
                'content': 'Hello, World!'          # The content itself
            }
        }
)

A new attribute:

- **cs** : This attribute contains the size of the content of the **con** attribute.

<img align="right" src="images/02/retrieve_contentInstance.png">


## Retrieve the latest created &lt;contentInstance> resource
This request will retrieve the latest data instance from the &lt;container>.

<p>
<div style="background-color:transparent;border:2px #b42025 solid;padding: 10px;width: calc(100% - 350px);">
    <div style="color:#b42025;"><b><i class="fa fa-info-circle" aria-hidden="true"></i>&nbsp; oneM2M</b></div>
    <div style="color:#545054;">
        The <em>resourceType</em> attribute is not necessary to provide in RETRIEVE requests, only in CREATE.
    </div>
</div>
</p>
 

In [ ]:
RETRIEVE (                                          # RETRIEVE request

    # Retrieve the latest <contentInstance> resource under the container
    to                      = cseBaseName + '/Notebook-AE/Container/la', 

    # Request Parameters
    originator              = 'Cmyself',            # Set the originator
    requestIdentifier       = '123',                # Unique request identifier
    releaseVersionIndicator = '3',                  # Release version indicator
)

<img align="right" src="images/02/update_container.png">


## Update the &lt;container> Resource

With this request we will set the *maxNumberOfInstances* (**mni**) attribute to a new value. This will restrict the number of &lt;contentInstance> resources for this &lt;container>.
 
<p>
<div style="background-color:transparent;border:2px #b42025 solid;padding: 10px;width: calc(100% - 350px);">
    <div style="color:#b42025;"><b><i class="fa fa-info-circle" aria-hidden="true"></i>&nbsp; oneM2M</b></div>
    <div style="color:#545054;">
        It is only necessary to provide the attributes we want to update in the request, not the whole resource.
    </div>
</div>
</p>


In [ ]:
UPDATE (                                            # UPDATE request

    # Target the container resource
    to                      = cseBaseName + '/Notebook-AE/Container', 

    # Request Parameters
    originator              = 'Cmyself',            # Set the originator
    requestIdentifier       = '123',                # Unique request identifier
    releaseVersionIndicator = '3',                  # Release version indicator
    
    # Request Body
    primitiveContent = 
        {
            'm2m:Container': {
                 'maxNrOfInstances': 10            # Set the maxNumberOfInstances to 10
            }
        }
)

The CSE returns the resource. 

 
<p>
<div style="background-color:transparent;border:2px #b42025 solid;padding: 10px;width: calc(100% - 350px);">
    <div style="color:#b42025;"><b><i class="fa fa-info-circle" aria-hidden="true"></i>&nbsp; oneM2M</b></div>
    <div style="color:#545054;">
    Also note the change of the <i>lastModificationTime</i> (lt) attribute. It is set to the time of the update.
    </div>
</div>
</p>
 
 


<img align="right" src="images/02/retrieve_container.png">

## Retrieve the &lt;container> Resource
Retrieve the &lt;container> resource to see all the changes and its current state.

In [ ]:
RETRIEVE (                                          # RETRIEVE request

    # Retrieve the container resource
    to                      = cseBaseName + '/Notebook-AE/Container', 

    # Request Parameters
    originator              = 'Cmyself',            # Set the originator
    requestIdentifier       = '123',                # Unique request identifier
    releaseVersionIndicator = '3',                  # Release version indicator
)

The CSE returns the resource.

<img align="right" src="images/02/delete_container.png">

## Delete the &lt;container Resource

The last request in this lecture removes the &lt;container> resource from the CSE's oneM2M resource tree.


In [ ]:
DELETE (                                            # DELETE request

    # Delete the container resource
    to                      = cseBaseName + '/Notebook-AE/Container', 

    # Request Parameters
    originator              = 'Cmyself',            # Set the originator
    requestIdentifier       = '123',                # Unique request identifier
    releaseVersionIndicator = '3',                  # Release version indicator
)

 <p>
<div style="background-color:transparent;border:2px #b42025 solid;padding: 10px;width: calc(100% - 350px);">
    <div style="color:#b42025;"><b><i class="fa fa-info-circle" aria-hidden="true"></i>&nbsp; oneM2M</b></div>
    <div style="color:#545054;">
When a resource is deleted then all its child resources, for example the &lt;contentInstance> resources in our exercise, are deleted as well.
    </div>
</div>
</p>